In [1]:
import azureml.core
print(azureml.core.VERSION)

1.3.0


In [2]:
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace 
import os 

ws = Workspace.from_config(auth=InteractiveLoginAuthentication(tenant_id=os.environ["AML_TENANT_ID"]))
ws

Workspace.create(name='pipelines', subscription_id='65a1016d-0f67-45d2-b838-b8f373d6d52e', resource_group='laobri-ml')

In [3]:
from azureml.core import Datastore
from azureml.core.compute import AmlCompute, ComputeTarget


compute_name = "cpu-compute3"
if not compute_name in ws.compute_targets :
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                min_nodes=0,
                                                                max_nodes=1)
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # Show the result
    print(compute_target.get_status().serialize())

In [4]:
from azureml.core.compute import AmlCompute 

compute_target = AmlCompute(ws, compute_name)
print(compute_target)

AmlCompute(workspace=Workspace.create(name='pipelines', subscription_id='65a1016d-0f67-45d2-b838-b8f373d6d52e', resource_group='laobri-ml'), name=cpu-compute3, id=/subscriptions/65a1016d-0f67-45d2-b838-b8f373d6d52e/resourceGroups/laobri-ml/providers/Microsoft.MachineLearningServices/workspaces/pipelines/computes/cpu-compute3, type=AmlCompute, provisioning_state=Succeeded, location=westus, tags=None)


In [5]:
from azureml.core import Datastore

datastore = ws.get_default_datastore()

In [6]:
e = ws.environments['AzureML-Tutorial']

In [7]:
type(e)

azureml.core.environment.Environment

In [8]:
[p for p in e.python.conda_dependencies.pip_packages]

['azureml-core==1.7.0',
 'azureml-defaults==1.7.0',
 'azureml-telemetry==1.7.0',
 'azureml-train-restclients-hyperdrive==1.7.0',
 'azureml-train-core==1.7.0',
 'azureml-widgets==1.7.0',
 'azureml-pipeline-core==1.7.0.post1',
 'azureml-pipeline-steps==1.7.0',
 'azureml-opendatasets==1.7.0.post1',
 'azureml-automl-core==1.7.0.post1',
 'azureml-automl-runtime==1.7.0.post1',
 'azureml-train-automl-client==1.7.0.post1',
 'azureml-train-automl-runtime==1.7.0.post1',
 'azureml-train-automl==1.7.0',
 'azureml-train==1.7.0',
 'azureml-sdk==1.7.0.post1',
 'azureml-interpret==1.7.0.post1',
 'azureml-tensorboard==1.7.0',
 'azureml-mlflow==1.7.0',
 'mlflow',
 'sklearn-pandas']

In [9]:
# True if the dependencies include that string at least once
True in ('sklearn' in p for p in e.python.conda_dependencies.pip_packages)

True

In [10]:
type(ws.environments.values)

builtin_function_or_method

In [27]:
def env_contains(e, deps) : 
    def env_contains_single(e, dep) : 
        conda_dependencies = e.python.conda_dependencies
        # Return True if pip-installed (`p` contains version, e.g., azureml-core==1.7.0.post1,  so can't just `dep in pip_packages`)
        if (dep in p for p in conda_dependencies.pip_packages) :
            return True
        # Return True if conda-installled
        if dep in conda_dependencies.conda_packages :
            return True
        return False
    return not (False in (env_contains_single(e, dep) for dep in deps))

def envs_containing(deps) : 
    for name in ws.environments :
        if env_contains(ws.environments[name], deps) : 
            yield name

list(envs_containing(['automl']))

['AzureML-TensorFlow-2.0-CPU',
 'AzureML-Chainer-5.1.0-GPU',
 'AzureML-TensorFlow-1.12-CPU',
 'AzureML-TensorFlow-2.0-GPU',
 'AzureML-PyTorch-1.1-CPU',
 'AzureML-PyTorch-1.1-GPU',
 'AzureML-PyTorch-1.2-CPU',
 'AzureML-Minimal',
 'AzureML-TensorFlow-1.10-CPU',
 'AzureML-PyTorch-1.3-CPU',
 'AzureML-PyTorch-1.0-CPU',
 'AzureML-Tutorial',
 'AzureML-PyTorch-1.2-GPU',
 'AzureML-Scikit-learn-0.20.3',
 'AzureML-AutoML',
 'AzureML-TensorFlow-1.13-CPU',
 'AzureML-TensorFlow-1.12-GPU',
 'AzureML-PyTorch-1.3-GPU',
 'AzureML-PyTorch-1.0-GPU',
 'AzureML-Chainer-5.1.0-CPU',
 'AzureML-PyTorch-1.4-GPU',
 'AzureML-TensorFlow-1.13-GPU',
 'AzureML-TensorFlow-1.10-GPU',
 'AzureML-PySpark-MmlSpark-0.15',
 'AzureML-Dask-CPU',
 'AzureML-Dask-GPU',
 'AzureML-PyTorch-1.5-CPU',
 'AzureML-PyTorch-1.5-GPU',
 'AzureML-Sidecar',
 'AzureML-Designer-CV-Transform',
 'AzureML-Designer-Score',
 'AzureML-Designer-PyTorch',
 'AzureML-Designer-CV',
 'AzureML-TensorFlow-2.1-GPU',
 'AzureML-TensorFlow-2.1-CPU',
 'AzureML-Vowp

^ Is a little better but still insufficient, as it seems to miss transient packages. For instance, the pipeline developed below relies on `pyarrow`, but you won't see that in the list below:

In [12]:
ws.environments['AzureML-Tutorial']

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "username": null
        },
        "enabled": false,
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "AzureML-Tutorial",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
            

In [13]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

aml_run_config = RunConfiguration()
aml_run_config.target = compute_target
from azureml.core import Environment

# Get curated environment
curated_environment = Environment.get(workspace=ws, name="AutoML-Tutorial")

USE_CURATED_ENV = True
if USE_CURATED_ENV :
    aml_run_config.environment = curated_environment
else:
    #Use conda_dependencies.yml to create a conda environment in the Docker image for execution
    aml_run_config.environment.python.user_managed_dependencies = False

    #Specify CondaDependencies obj, add necessary packages
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn', 'pyarrow'], 
        pip_packages=['azureml-sdk[automl,explain]', 'azureml-dataprep[fuse,pandas]'], 
        pin_sdk_version=False)

## Step 0: Grab an open dataset and register it

This is baseline data. If the `Dataset` does not exist, create and register it. Not a part of the Pipeline.

In [14]:
from azureml.core import Dataset

if not 'titanic_ds' in ws.datasets.keys() :
    # create a TabularDataset from Titanic training data
    web_paths = ['https://dprepdata.blob.core.windows.net/demo/Titanic.csv',
                 'https://dprepdata.blob.core.windows.net/demo/Titanic2.csv']
    titanic_ds = Dataset.Tabular.from_delimited_files(path=web_paths)

    titanic_ds.register(workspace = ws,
                                     name = 'titanic_ds',
                                     description = 'new titanic training data',
                                     create_new_version = True)

titanic_ds = Dataset.get_by_name(ws, 'titanic_ds')

In [15]:
type(titanic_ds)

azureml.data.tabular_dataset.TabularDataset

## Step 1: Dataprep

In [16]:
%%writefile dataprep.py
from azureml.core import Run

import pandas as pd 
import numpy as np 
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
import argparse

RANDOM_SEED=42

def prepare_age(df):
    # Fill in missing Age values from distribution of present Age values 
    mean = df["Age"].mean()
    std = df["Age"].std()
    is_null = df["Age"].isnull().sum()
    # compute enough (== is_null().sum()) random numbers between the mean, std
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = df["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    df["Age"] = age_slice
    df["Age"] = df["Age"].astype(int)
    
    # Quantize age into 5 classes
    df['Age_Group'] = pd.qcut(df['Age'],5, labels=False)
    df.drop(['Age'], axis=1, inplace=True)
    return df

def prepare_fare(df):
    df['Fare'].fillna(0, inplace=True)
    df['Fare_Group'] = pd.qcut(df['Fare'],5,labels=False)
    df.drop(['Fare'], axis=1, inplace=True)
    return df 

def prepare_genders(df):
    genders = {"male": 0, "female": 1, "unknown": 2}
    df['Sex'] = df['Sex'].map(genders)
    df['Sex'].fillna(2, inplace=True)
    df['Sex'] = df['Sex'].astype(int)
    return df

def prepare_embarked(df):
    df['Embarked'].replace('', 'U', inplace=True)
    df['Embarked'].fillna('U', inplace=True)
    ports = {"S": 0, "C": 1, "Q": 2, "U": 3}
    df['Embarked'] = df['Embarked'].map(ports)
    return df
    
parser = argparse.ArgumentParser()
parser.add_argument('--output_path', dest='output_path', required=True)
args = parser.parse_args()
    
titanic_ds = Run.get_context().input_datasets['titanic_ds']
df = titanic_ds.to_pandas_dataframe().drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
df = prepare_embarked(prepare_genders(prepare_fare(prepare_age(df))))

os.makedirs(os.path.dirname(args.output_path), exist_ok=True)
pq.write_table(pa.Table.from_pandas(df), args.output_path)

print(f"Wrote test to {args.output_path} and train to {args.output_path}")

Overwriting dataprep.py


In [17]:
from azureml.pipeline.core import PipelineData

prepped_data_path = PipelineData("titanic_train", datastore).as_dataset()

In [18]:
from azureml.pipeline.steps import PythonScriptStep

dataprep_step = PythonScriptStep(
    name="dataprep", 
    script_name="dataprep.py", 
    compute_target=compute_target, 
    runconfig=aml_run_config,
    arguments=["--output_path", prepped_data_path],
    inputs=[titanic_ds.as_named_input("titanic_ds")],
    outputs=[prepped_data_path],
    allow_reuse=True
)

### Step 2: Train with AutoMLStep

In [19]:
from azureml.train.automl import AutoMLConfig

prepped_data_potds = prepped_data_path.parse_parquet_files(file_extension=None)

# Change iterations to a reasonable number (50) to get better accuracy
automl_settings = {
    "iteration_timeout_minutes" : 10,
    "iterations" : 10,
    #"experiment_timeout_hours" : .1,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(task = 'classification',
                             path = '.',
                             debug_log = 'automated_ml_errors.log',
                             compute_target = compute_target,
                             run_configuration = aml_run_config,
                             featurization = 'auto',
                             training_data = prepped_data_potds,
                             label_column_name = 'Survived',
                             **automl_settings)
                             
print("AutoML config created.")

AutoML config created.


In [20]:
from azureml.pipeline.core import TrainingOutput
from azureml.pipeline.steps import AutoMLStep

metrics_data = PipelineData(name='metrics_data',
                           datastore=datastore,
                           pipeline_output_name='metrics_output',
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='best_model_data',
                           datastore=datastore,
                           pipeline_output_name='model_output',
                           training_output=TrainingOutput(type='Model'))


train_step = AutoMLStep(name='AutoML_Classification',
                                 automl_config=automl_config,
                                 passthru_automl_config=False,
                                 outputs=[metrics_data,model_data],
                                 allow_reuse=True)
print("train_step created.")

train_step created.


## Step 3: Register the model

In [21]:
%%writefile register_model.py
from azureml.core.model import Model, Dataset
from azureml.core.run import Run, _OfflineRun
from azureml.core import Workspace
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--model_name", required=True)
parser.add_argument("--model_path", required=True)
args = parser.parse_args()

print(f"model_name : {args.model_name}")
print(f"model_path: {args.model_path}")

run = Run.get_context()
ws = Workspace.from_config() if type(run) == _OfflineRun else run.experiment.workspace

model = Model.register(workspace=ws,
                       model_path=args.model_path,
                       model_name=args.model_name)

print("Registered version {0} of model {1}".format(model.version, model.name))


Overwriting register_model.py


In [22]:
from azureml.pipeline.core.graph import PipelineParameter

# The model name with which to register the trained model in the workspace.
model_name = PipelineParameter("model_name", default_value="TitanicSurvival")

register_step = PythonScriptStep(script_name="register_model.py",
                                       name="register_model",
                                       allow_reuse=False,
                                       arguments=["--model_name", model_name, "--model_path", model_data],
                                       inputs=[model_data],
                                       compute_target=compute_target,
                                       runconfig=aml_run_config)

## Submit it

In [23]:
from azureml.core import Experiment

if not 'titanic_automl' in ws.experiments.keys() :
    Experiment(ws, 'titanic_automl')
experiment = ws.experiments['titanic_automl']

In [24]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, [dataprep_step, train_step, register_step])

In [25]:
run = experiment.submit(pipeline, show_output=True)

Created step dataprep [f8a165bc][854ee7a9-c85b-4a9d-9872-b8af61e18ea4], (This step will run and generate new outputs)
Created step AutoML_Classification [4c048bc8][c7b9d2c3-b5a4-4eff-8904-aee65b1676bd], (This step will run and generate new outputs)
Created step register_model [b468f480][579f3abc-bf4f-46c7-a749-500e218c28b6], (This step will run and generate new outputs)
Submitted PipelineRun b07848c5-644c-41f5-be07-1de561a4c836
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/titanic_automl/runs/b07848c5-644c-41f5-be07-1de561a4c836?wsid=/subscriptions/65a1016d-0f67-45d2-b838-b8f373d6d52e/resourcegroups/laobri-ml/workspaces/pipelines


In [26]:
import time 

start = time.time()
run.wait_for_completion()
duration = start - time.time()
print(f"Use configured environment? {USE_CURATED_ENV} Duration in seconds {duration}")

PipelineRunId: b07848c5-644c-41f5-be07-1de561a4c836
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/titanic_automl/runs/b07848c5-644c-41f5-be07-1de561a4c836?wsid=/subscriptions/65a1016d-0f67-45d2-b838-b8f373d6d52e/resourcegroups/laobri-ml/workspaces/pipelines
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 4e8dd414-da72-49da-85ef-b3f21506ba81
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/titanic_automl/runs/4e8dd414-da72-49da-85ef-b3f21506ba81?wsid=/subscriptions/65a1016d-0f67-45d2-b838-b8f373d6d52e/resourcegroups/laobri-ml/workspaces/pipelines
StepRun( dataprep ) Status: NotStarted
StepRun( dataprep ) Status: Running

Streaming azureml-logs/20_image_build_log.txt


WARNING - Retrying (Retry(total=2, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x110f99650>: Failed to establish a new connection: [Errno 60] Operation timed out')': /history/v1.0/subscriptions/65a1016d-0f67-45d2-b838-b8f373d6d52e/resourceGroups/laobri-ml/providers/Microsoft.MachineLearningServices/workspaces/pipelines/experiments/titanic_automl/runs/4e8dd414-da72-49da-85ef-b3f21506ba81/details


2020/06/12 22:39:47 Downloading source code...
2020/06/12 22:39:49 Finished downloading source code
2020/06/12 22:39:49 Creating Docker network: acb_default_network, driver: 'bridge'
2020/06/12 22:39:49 Successfully set up Docker network: acb_default_network
2020/06/12 22:39:49 Setting up Docker configuration...
2020/06/12 22:39:50 Successfully set up Docker configuration
2020/06/12 22:39:50 Logging in to registry: pipelines5112a485.azurecr.io
2020/06/12 22:39:51 Successfully logged into pipelines5112a485.azurecr.io
2020/06/12 22:39:51 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/06/12 22:39:51 Scanning for dependencies...
2020/06/12 22:39:52 Successfully scanned dependencies
2020/06/12 22:39:52 Launching container with name: acb_step_0
Sending build context to Docker daemon  60.93kB

Step 1/15 : FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04@sha256:a1b514f3ba884b9a7695cbba5638933ddaf222e8ce3e8c81e8cdf8616

  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3923 sha256=a03a989eb596bc7269c4bdb6572d86e1b95f513da74f43f95acfacc9b9786baf
  Stored in directory: /root/.cache/pip/wheels/e2/1d/52/535a274b9c2ce7d4064838f2bdb62013801281ef7d7f21e2ee
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78530 sha256=547742b0d0436bc31f964dd0564a33a4a238bada6767d989ed80b508fa3d9b95
  Stored in directory: /root/.cache/pip/wheels/09/84/74/d2b4feb9ac9488bc83c475cb2cbe8e8b7d9cea8320d32f3787
  Created wheel for liac-arff: filename=liac_arff-2.4.0-py3-none-any.whl size=13333 sha256=34ab84b378959fba64722cba99f0e2f345f9764cc978a78e7e61d710b98c7509
  Stored in directory: /root/.cache/pip/wheels/ba/2a/e1/6f7be2e2ea150e2486bff64fd6f0670f4f35f4c8f31c819fb8
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10503 sha256=092b8347044a9334228f8eb8f477793dfcf735a2568eea1766569748231b534e
  Stored in directory: /root/.cache/pip/wheels/21/5c/83/1dd7


#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_6162e2d2e32d3224150f0271123ffbfb
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container 193aa5e238fe
 ---> af809a2eed2c
Step 9/15 : ENV PATH /azureml-envs/azureml_6162e2d2e32d3224150f0271123ffbfb/bin:$PATH
 ---> Running in 2d055ce5607c
Removing intermediate container 2d055ce5607c
 ---> 9cee76c9b4c2
Step 10/15 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_6162e2d2e32d3224150f0271123ffbfb
 ---> Running in 9ab2da76f2c4
Removing intermediate container 9ab2da76f2c4
 ---> 118e89cf72e9
Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_6162e2d2e32d3224150f0271123ffbfb/lib:$LD_LIBRARY_PATH
 ---> Running in 00587600357f
Removing intermediate container 00587600357f
 ---> d87d2957734c
Step 12/15 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> 330d1505ef8b
Step 13/15 : RUN if 

WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(60, 'ETIMEDOUT')"))': /azureml/ExperimentRun/dcid.4e8dd414-da72-49da-85ef-b3f21506ba81/azureml-logs/20_image_build_log.txt?sv=2019-02-02&sr=b&sig=wQdISSxudyRccBgnjQqHhMPoPDG5wCPEfEBDblzItfo%3D&st=2020-06-12T22%3A35%3A59Z&se=2020-06-13T06%3A45%3A59Z&sp=r



Streaming azureml-logs/55_azureml-execution-tvmps_3e11f9e75c8a8d7bc0659f3abd4c450373af1d0600864c784b3c97723b044749_d.txt
2020-06-12T22:46:35Z Executing 'Copy ACR Details file' on 10.0.0.4
2020-06-12T22:46:35Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
2020-06-12T22:46:35Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2020-06-12T22:46:35Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_080bb68af85f96b85eb7cff522629683
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
6504d449e70c: Pulling fs layer
4e38f320d0d4: Pulling fs layer
b0a763e8ee03: Pulling fs layer
11917a028ca4: Pulling fs layer
a6c378d11cbf: Pulling fs layer
6cc007ad9140: Pulling fs layer
6c1698a608f3: Pulling fs layer
c7c110b9ca73: Pulling fs layer
a1eed15b592d: Pulling fs layer
412964b3befe: Pulling fs layer
f9efed77ad39: Pulling fs layer
81a9cedab415: Pulling

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with ModuleNotFoundError: No module named 'pyarrow'",
        "detailsUri": "https://aka.ms/azureml-known-errors",
        "details": [],
        "debugInfo": {
            "type": "ModuleNotFoundError",
            "message": "No module named 'pyarrow'",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/pipelines/azureml/4e8dd414-da72-49da-85ef-b3f21506ba81/mounts/workspaceblobstore/azureml/4e8dd414-da72-49da-85ef-b3f21506ba81/azureml-setup/context_manager_injector.py\", line 148, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_6162e2d2e32d3224150f0271123ffbfb/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azureml_6162e2d2e32d3224150f0271123ffbfb/lib/python3.6/r

In [ ]:
# automl_run = next(r for r in run.get_children() if r.name == 'AutoML_Classification')
# outputs = automl_run.get_outputs()
# metrics = outputs['default_metrics_AutoML_Classification']
# model = outputs['default_model_AutoML_Classification']

# metrics.get_port_data_reference().download('.')
# model.get_port_data_reference().download('.')

In [ ]:
#metrics

In [ ]:
#type(model)

In [ ]:
#model

In [ ]:
run

In [ ]:
metrics_output = run.get_pipeline_output('metrics_output')
num_file_downloaded = metrics_output.download('.', show_progress=True)
import pandas as pd
# import numpy as np
import json
with open(metrics_output._path_on_datastore) as f:
   metrics_output_result = f.read()
   
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

In [ ]:
metrics_output

In [ ]:
num_file_downloaded

In [ ]:
run

In [ ]:
metrics_output

In [ ]:
type(metrics_output)